In [1]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
import base64
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from io import BytesIO


app = dash.Dash(__name__ ,suppress_callback_exceptions=True)
app.css.append_css({'external_url': '/assets/styles.css'})



C:\Users\ASUS\AppData\Local\Temp\ipykernel_23264\4062153994.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ASUS\AppData\Local\Temp\ipykernel_23264\4062153994.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Shivam

In [2]:
import pickle
# Load movies_dict and tv_series_dict
movies_dict = pickle.load(open('./Shivam/movies_dict.pkl','rb'))
tv_series_dict = pickle.load(open('./Shivam/tv_series_dict.pkl','rb'))
genre_year_scatter_figures_dict_movies = pickle.load(open('./Shivam/genre_year_scatter_figures_dict_movies.pkl','rb'))

In [3]:
# Revenue Analysis for movies_dict
movies_revenue_df = {}
for genre, df in movies_dict.items():
    df = df.dropna(subset=['Gross Revenue'])
    df = df.reset_index()
    movies_revenue_df[genre] = df

# Cleaning Gross Revenue for movies_dict
for genre, genre_df in movies_revenue_df.items():
    # Clean the 'Gross Revenue' column for the current genre's DataFrame
    genre_df['Gross Revenue(million $)'] = (genre_df['Gross Revenue']/1000000).round(2)

    # Drop Column Gross Revenue
    genre_df = genre_df.drop(columns=['Gross Revenue'])
    genre_df = genre_df.reset_index()
    # Update the DataFrame in the dictionary
    movies_revenue_df[genre] = genre_df

# Revenue Analysis
tv_series_revenue_df = {}
for genre, df in tv_series_dict.items():
    df = df.dropna(subset=['Gross Revenue'])
    df = df.reset_index()
    tv_series_revenue_df[genre] = df

# Cleaning Gross Revenue
for genre, genre_df in tv_series_revenue_df.items():
    # Clean the 'Gross Revenue' column for the current genre's DataFrame
    genre_df['Gross Revenue(million $)'] = (genre_df['Gross Revenue'].str.replace(',', '').astype(float)/1000000).round(2)

    # Drop Column Gross Revenue
    genre_df = genre_df.drop(columns=['Gross Revenue'])
    genre_df = genre_df.reset_index()
    # Update the DataFrame in the dictionary
    tv_series_revenue_df[genre] = genre_df

def getGenreOverYearsScatter(release_year,line):
    counts_per_year = release_year.value_counts().sort_index().to_frame()
    counts_per_year.reset_index(inplace=True)
    # Filter out movies released before 1980
    counts_per_year = counts_per_year[counts_per_year['Release Year'] >= 1980]
    fig = go.Figure(data = go.Scatter(x = counts_per_year['Release Year'], y = counts_per_year['count'],line=line))
    # Update layout
    fig.update_layout(title='TV Shows Count Per Year',
                    xaxis_title='Year',
                    yaxis_title='TV Shows Count')
    return fig
# Dictionary to store figures for each genre
genre_year_scatter_figures_dict = {}

# Iterate through each genre DataFrame in the tv_series_dict dictionary
for genre, df in tv_series_dict.items():
    # Run the function for the current genre DataFrame and store the resulting figure
    genre_year_scatter_figures_dict[genre] = getGenreOverYearsScatter(df['Release Year'],dict(color='#008080'))

# Dictionary to store figures for each genre for movies
genre_year_scatter_figures_dict_movies = {}

# Iterate through each genre DataFrame in the movies_dict dictionary
for genre, df in movies_dict.items():
    # Run the function for the current genre DataFrame and store the resulting figure
    genre_year_scatter_figures_dict_movies[genre] = getGenreOverYearsScatter(df['Release Year'],dict(color='#ffa500'))


def getGenreOverYearsBar(dictionary):
    # Define the range of years
    years = list(range(1980, 2022,4))
    count_year_wise_dict = {str(year): {} for year in years}
    color_list = [f'#{0x0000FF + i:06X}' for i in range(0, 256, 25)]
    # Iterate over the years and populate the dictionary
    for year in years:
        for genre in dictionary.keys():
            count_year_wise_dict[str(year)][genre] = {}
    print(years)
    genre_count_year = {}
    for year in years:
        genre_count = {}
        for genre, df in dictionary.items():
            genre_count[genre] = len(df[df['Release Year'] == year])
        temp_df = pd.DataFrame.from_dict(genre_count, orient='index').reset_index()
        temp_df.columns = ['Genre', 'Count']
        # Sort the DataFrame by the 'Count' column in descending order
        temp_df = temp_df.sort_values(by='Count')
        temp_df = temp_df.reset_index(drop=True)
        # Assign colors to each genre
        num_colors = len(color_list)
        color_index = 0
        genre_colors = {}

        for genre in temp_df['Genre'].unique():
            genre_colors[genre] = color_list[color_index]
            color_index = (color_index + 1) % num_colors

        # Map colors to genres
        temp_df['Color'] = temp_df['Genre'].map(genre_colors)
        genre_count_year[str(year)] = temp_df

    return genre_count_year
genre_count_year_tvseries = getGenreOverYearsBar(tv_series_dict)


def getGenreOverYearsBarMovies(dictionary):
    # Define the range of years
    years = list(range(1980, 2022,4))
    count_year_wise_dict = {str(year): {} for year in years}
    color_list = [f'#{0x0000FF + i:06X}' for i in range(0, 256, 25)]
    # Iterate over the years and populate the dictionary
    for year in years:
        for genre in dictionary.keys():
            count_year_wise_dict[str(year)][genre] = {}
    genre_count_year = {}
    for year in years:
        genre_count = {}
        for genre, df in dictionary.items():
            genre_count[genre] = len(df[df['Release Year'] == year])
        temp_df = pd.DataFrame.from_dict(genre_count, orient='index').reset_index()
        temp_df.columns = ['Genre', 'Count']

        # Assign colors to each genre
        num_colors = len(color_list)
        color_index = 0
        genre_colors = {}

        for genre in temp_df['Genre'].unique():
            genre_colors[genre] = color_list[color_index]
            color_index = (color_index + 1) % num_colors

        # Map colors to genres
        temp_df['Color'] = temp_df['Genre'].map(genre_colors)
        genre_count_year[str(year)] = temp_df
        genre_count_year[str(year)] = genre_count_year[str(year)].sort_values(by='Count')
    return genre_count_year

genre_count_year_movies = getGenreOverYearsBarMovies(movies_dict)

[1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]


In [4]:
# 1. Racing Chart Movies
fig_racing_chart_movies = go.Figure(
    data=[
        go.Bar(
            x=genre_count_year_movies['1980']['Count'],
            y=genre_count_year_movies['1980']['Genre'],
            orientation='h',
            text=genre_count_year_movies['1980']['Count'],
            width=0.9,
            marker={'color': genre_count_year_movies['1980']['Count'],'colorscale': 'Blues','line':dict(color='rgba(0,0,0,0.5)', width=5)},
        )
    ],
    layout=go.Layout(
        xaxis=dict(range=[0, 1600], autorange=False,tickfont=dict(size=16, color='white'), title=dict(text='Count of Movies', font=dict(size=16, color='white'))),
        yaxis=dict(range=[-1, 10],automargin=True, autorange=False, tickfont=dict(size=16, color='white'),title=dict(text='Genres', font=dict(size=16, color='white'))),
        title=dict(text=f'Genre Wise Movie Count: 1980', font=dict(size=28, color='white'), x=0.5, xanchor='center'),
        font=dict(size=15),
        margin = {'pad': 13},
        plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)',
        height=500,
        updatemenus=[dict(
            type="buttons",
            direction="left",
            pad={"r": 20, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=-0.09,
            yanchor="top",
            buttons=[dict(
                label="      Play     ",
                method="animate",
                args=[
                    None,
                    {
                        "frame": {"duration": 1000, "redraw": True},
                        "transition": {"duration": 250, "easing": "quadratic-in-out"}
                    }
                ],
            )]
        )]
    ),
    frames=[
        go.Frame(
            data=[
                go.Bar(
                    x=value['Count'],
                    y=value['Genre'],
                    orientation='h',
                    text=value['Count'],
                    marker={'color': value['Count'],'colorscale': 'Blues','line':dict(color='rgba(0,0,0,0.5)', width=5)},
                    width=0.9,
                )
            ],
            layout=go.Layout(
                xaxis=dict(range=[0, 1600], autorange=False,tickfont=dict(size=16, color='white'), title=dict(text='Count of Movies', font=dict(size=16, color='white'))),
                yaxis=dict(range=[-1, 10],automargin=True, autorange=False, tickfont=dict(size=16, color='white'),title=dict(text='Genres', font=dict(size=16, color='white'))),
                title=dict(text=f'Genre Wise Movie Count: {key}', font=dict(size=28, color='white'), x=0.5, xanchor='center'),
                font=dict(size=15),
                margin = {'pad': 13},
                plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
                paper_bgcolor='rgba(0,0,0,0)'
                )
        )
        for key, value in genre_count_year_movies.items()
    ]
)

# 2. Racing Chart TV series
fig_racing_chart_tvseries = go.Figure(
    data=[
        go.Bar(
            x=genre_count_year_tvseries['1980']['Count'],
            y=genre_count_year_tvseries['1980']['Genre'],
            orientation='h',
            text=genre_count_year_tvseries['1980']['Count'],
            width=0.9,
            marker={'color': genre_count_year_tvseries['1980']['Count'],'colorscale': 'Blues','line':dict(color='rgba(0,0,0,0.5)', width=5)},
        )
    ],
    layout=go.Layout(
        xaxis=dict(range=[0, 300], autorange=False,tickfont=dict(size=16, color='white'), title=dict(text='Count of TV Series', font=dict(size=16, color='white'))),
        yaxis=dict(range=[-1, 10], autorange=False, tickfont=dict(size=16, color='white'),title=dict(text='Genres', font=dict(size=16, color='white'))),
        title=dict(text=f'Genre Wise TV Series Count: 1980', font=dict(size=28, color='white'), x=0.5, xanchor='center'),
        font=dict(size=13),
        margin = {'pad': 20},
        plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)',
        height=500,
        updatemenus=[dict(
            type="buttons",
            direction="left",
            pad={"r": 20, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=-0.1,
            yanchor="top",
            buttons=[dict(
                label="      Play     ",
                method="animate",
                args=[
                    None,
                    {
                        "frame": {"duration": 1000, "redraw": True},
                        "transition": {"duration": 250, "easing": "quadratic-in-out"}
                    }
                ],
            )]
        )]
    ),
    frames=[
        go.Frame(
            data=[
                go.Bar(
                    x=value['Count'],
                    y=value['Genre'],
                    orientation='h',
                    text=value['Count'],
                    marker={'color': value['Count'],'colorscale': 'Blues','line':dict(color='rgba(0,0,0,0.5)', width=5)},
                    width=0.9,
                )
            ],
            layout=go.Layout(
                xaxis=dict(range=[0, 300], autorange=False,tickfont=dict(size=16, color='white'), title=dict(text='Count of TV Series', font=dict(size=16, color='white'))),
                yaxis=dict(range=[-1, 10], autorange=False, tickfont=dict(size=16, color='white'),title=dict(text='Genres', font=dict(size=16, color='white'))),
                title=dict(text=f'Genre Wise TV Series Count: {key}', font=dict(size=28, color='white'), x=0.5, xanchor='center'),
                font=dict(size=13),
                margin = {'pad': 20},
                plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
                paper_bgcolor='rgba(0,0,0,0)'
                )
        )
        for key, value in genre_count_year_tvseries.items()
    ]
)

# Combine both figures into a single dcc.Graph
combined_graph = dcc.Graph(
    id='combined-graph',
    figure={
        'data': fig_racing_chart_movies['data'] + fig_racing_chart_tvseries['data'],
        'layout': fig_racing_chart_movies['layout']
    }
)

# 3. Total Count of Movies and TV Series
# Define callback to update the charts based on selected genre
def getFig1(selected_genre):
    # Create empty figure with subplots
    fig = go.Figure()

    # Add traces for movies
    trace_movies = genre_year_scatter_figures_dict_movies[selected_genre]['data'][0]  # Assuming only one trace per figure
    trace_movies['name'] = 'Movies'
    # Increase the width of markers
    trace_movies['marker']['size'] = 15
    # Set line color for movies
    trace_movies['line']['color'] = '#f5f5f5'  # Light gray color

    # Add traces for TV shows
    trace_tv = genre_year_scatter_figures_dict[selected_genre]['data'][0]  # Assuming only one trace per figure
    trace_tv['name'] = 'TV Series'
    # Increase the width of markers
    trace_tv['marker']['size'] = 15
    # Set line color for TV shows
    trace_tv['line']['color'] = '#01abd2'  # Light blue color

    fig.add_trace(trace_movies)
    fig.add_trace(trace_tv)
    

    # Define layout
    layout = go.Layout(
        title=f'{selected_genre} - TV Shows and Movies Count Per Year',
        xaxis={'title': 'Year', 'tickfont': {'color': 'white'}},  # Black tick labels
        yaxis={'title': 'Count', 'tickfont': {'color': 'white'}},  # Black tick labels
        showlegend=True,
        plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)',  # Yellow paper color
        font=dict(color='white')
    )
    fig.update_layout(layout)
    return fig

# 4. Rating Violin Chart
def getFig2(selected_genre):
    # Filter data for TV shows
    tv_data = tv_series_dict[selected_genre]

    # Filter data for movies
    movie_data = movies_dict[selected_genre]
    
    # Create subplots
    fig = go.Figure()
    
    fig.add_trace(go.Violin(y=movie_data['Rating'],
                            name='Movie',
                            box_visible=True,
                            meanline_visible=True,
                            line_color='#ffa500',  # Orange color for movie violin plot
                            fillcolor='#f5f5f5')  # Light yellow fill color for movie violin plot
    )
    fig.add_trace(go.Violin(y=tv_data['Rating'],
                            name='TV Series',
                            box_visible=True,
                            meanline_visible=True,
                            line_color='#008080',  # Teal color for TV series violin plot
                            fillcolor='#01abd2')  # Steel blue fill color for TV series violin plot
    )

    # Update layout
    fig.update_layout(
        title=f"Rating Distribution for {selected_genre}",
        xaxis_title='Type (Movies/TV Shows)',
        yaxis_title='Rating',
        plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)',  # Yellow paper color
        font=dict(color='white'),  # Black font color for better contrast,
        yaxis=dict(range=[0, 10])
    )

    return fig

# 5.Genre - IMDB Rating Over Years
def getFig3(selected_genre):
    fig = go.Figure()
        
    tv_data = tv_series_dict[selected_genre]
    tv_yearly_total_rating_df = tv_data.groupby('Release Year')['Rating'].sum().reset_index()

    movie_data = movies_dict[selected_genre]
    movie_yearly_total_rating_df = movie_data.groupby('Release Year')['Rating'].sum().reset_index()

    # Add trace for movies
    fig.add_trace(go.Scatter(x=movie_yearly_total_rating_df['Release Year'], y=movie_yearly_total_rating_df['Rating'], mode='lines', name='Movie', line=dict(color='#f5f5f5')))  # Orange color for movies

    # Add trace for TV series
    fig.add_trace(go.Scatter(x=tv_yearly_total_rating_df['Release Year'], y=tv_yearly_total_rating_df['Rating'], mode='lines', name='TV Series', line=dict(color='#01abd2')))  # Teal color for TV series
    
    # Update layout
    fig.update_layout(
        title='Total Rating Over Years',
        xaxis_title='Year',
        yaxis_title='Total Rating',
        plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)',  # Yellow paper color
        font=dict(color='white'), 
    )

    return fig

# 6. Genre - IMDB Average Rating Over Years
# Define function to get genre rating line chart
def getFig4(selected_genre):
    fig = go.Figure()
        
    tv_data = tv_series_dict[selected_genre]
    tv_yearly_total_rating_df = tv_data.groupby('Release Year')['Rating'].median().reset_index()

    movie_data = movies_dict[selected_genre]
    movie_yearly_total_rating_df = movie_data.groupby('Release Year')['Rating'].median().reset_index()

    # Add trace for movies
    fig.add_trace(go.Scatter(x=movie_yearly_total_rating_df['Release Year'], y=movie_yearly_total_rating_df['Rating'], mode='lines', name='Movie', line=dict(color='#f5f5f5')))  # Orange color for movies

    # Add trace for TV series
    fig.add_trace(go.Scatter(x=tv_yearly_total_rating_df['Release Year'], y=tv_yearly_total_rating_df['Rating'], mode='lines', name='TV Series', line=dict(color='#01abd2')))  # Teal color for TV series
    
    # Update layout
    fig.update_layout(
        title='Median Rating Over Years',
        xaxis_title='Year',
        yaxis_title='Median Rating',
        plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)',  # Yellow paper color
        font=dict(color='white')  # Black font color for better contrast,
    )

    return fig

# 7. Genre - IMDB Average Runtime Over Years
# Define function to get genre runtime line chart
def getFig5(selected_genre):
    fig = go.Figure()
    
    tv_data = tv_series_dict[selected_genre]
    tv_yearly_median_runtime_df = tv_data.groupby('Release Year')['Runtime'].median().reset_index()

    movie_data = movies_dict[selected_genre]
    movies_yearly_median_runtime_df = movie_data.groupby('Release Year')['Runtime'].median().reset_index()

    # Add trace for movies
    fig.add_trace(go.Scatter(x=movies_yearly_median_runtime_df['Release Year'], y=movies_yearly_median_runtime_df['Runtime'], mode='lines', name='Movies', line=dict(color='#f5f5f5')))  # Orange color for movies

    # Add trace for TV series
    fig.add_trace(go.Scatter(x=tv_yearly_median_runtime_df['Release Year'], y=tv_yearly_median_runtime_df['Runtime'], mode='lines', name='TV Series', line=dict(color='#01abd2')))  # Teal color for TV series
    
    # Update layout
    fig.update_layout(
        title='Median Runtime of Movies and TV Shows every Year',
        xaxis_title='Year',
        yaxis_title='Median Runtime (in minutes)',
        plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)',  # Yellow paper color
        font=dict(color='white'),  # Black font color for better contrast
        yaxis=dict(range=[0, 150])
    )

    return fig
# 6. Genre - IMDB Total Gross Revenue Over Years
def getFig6(selected_genre):
    tv_data = tv_series_revenue_df[selected_genre][['Release Year', 'Gross Revenue(million $)']]
    tv_data = tv_data.dropna(subset=['Gross Revenue(million $)'])
    tv_data = tv_data.reset_index()

    movie_data = movies_revenue_df[selected_genre][['Release Year', 'Gross Revenue(million $)']]
    movie_data = movie_data.dropna(subset=['Gross Revenue(million $)'])
    movie_data = movie_data.reset_index()
    
    # Group by 'Release Year' and sum 'Gross Revenue(million $)' for each year
    tv_grouped_data = tv_data.groupby('Release Year', observed=False)['Gross Revenue(million $)'].sum().reset_index()
    movie_grouped_data = movie_data.groupby('Release Year', observed=False)['Gross Revenue(million $)'].sum().reset_index()
    
    # Create the line plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=tv_grouped_data['Release Year'],
        y=tv_grouped_data['Gross Revenue(million $)'],
        mode='lines+markers',
        name='TV Series'
    ))
    fig.add_trace(go.Scatter(
        x=movie_grouped_data['Release Year'],
        y=movie_grouped_data['Gross Revenue(million $)'],
        mode='lines+markers',
        name='Movies'
    ))

    # Update layout
    fig.update_layout(
        title=f'Total Gross Revenue every Release Year for {selected_genre} Genre',
        xaxis=dict(title='Release Year'),
        yaxis=dict(title='Total Gross Revenue (million $)'),
        plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)'  # Yellow paper color
    )
    
    return fig


def getFig7(selected_genre):
    tv_data = tv_series_revenue_df[selected_genre][['Release Year', 'Gross Revenue(million $)']]
    tv_data = tv_data.dropna(subset=['Gross Revenue(million $)'])
    tv_data = tv_data.reset_index()

    movie_data = movies_revenue_df[selected_genre][['Release Year', 'Gross Revenue(million $)']]
    movie_data = movie_data.dropna(subset=['Gross Revenue(million $)'])
    movie_data = movie_data.reset_index()
    
    # Group by 'Release Year' and sum 'Gross Revenue(million $)' for each year
    tv_grouped_data = tv_data.groupby('Release Year', observed=False)['Gross Revenue(million $)'].median().reset_index()
    movie_grouped_data = movie_data.groupby('Release Year', observed=False)['Gross Revenue(million $)'].median().reset_index()
    
    # Create the line plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=movie_grouped_data['Release Year'],
        y=movie_grouped_data['Gross Revenue(million $)'],
        mode='lines+markers',
        name='Movies'
    ))
    fig.add_trace(go.Scatter(
        x=tv_grouped_data['Release Year'],
        y=tv_grouped_data['Gross Revenue(million $)'],
        mode='lines+markers',
        name='TV Series'
    ))

    # Update layout
    fig.update_layout(
        title=f'Average Gross Revenue every Release Year for {selected_genre} Genre',
        xaxis=dict(title='Release Year'),
        yaxis=dict(title='Total Gross Revenue (million $)'),
       plot_bgcolor='rgba(0,0,0,0)',  # Yellow background color
        paper_bgcolor='rgba(0,0,0,0)'  # Yellow paper color
    )
    
    return fig


# Define callback to update all charts based on selected genre
@app.callback(
    [Output('fig1', 'figure'),
     Output('fig2', 'figure'),
     Output('fig3', 'figure'),
     Output('fig4', 'figure'),
     Output('fig5', 'figure')],
    [Input('genre-dropdown', 'value')]
)
def update_charts(selected_genre):
    fig1 = getFig1(selected_genre)
    fig2 = getFig2(selected_genre)
    fig3 = getFig3(selected_genre)
    fig4 = getFig4(selected_genre)
    fig5 = getFig5(selected_genre)
    fig6 = getFig6(selected_genre)
    fig7 = getFig7(selected_genre)

    return fig1, fig2, fig3, fig4, fig5



# **Vipul and Vishnu Tab**

In [5]:
import pickle
with open('Vipul/figV1.pkl', 'rb') as file:
    figV1 = pickle.load(file)

with open('Vipul/time_traces.pkl', 'rb') as file:
    time_traces = pickle.load(file)

with open('Vipul/df_top_critic_pntve.pkl', 'rb') as file:
    df_top_critic_pntve = pickle.load(file)

with open('Vipul/top_actors_by_genre.pkl', 'rb') as file:
    top_actors_by_genre = pickle.load(file)

with open('Vipul/dictionary_top_5_directors_each_genre.pkl', 'rb') as file:
    dictionary_top_5_directors_each_genre = pickle.load(file)

with open('Vipul/indices_top_5_directors_each_genre.pkl', 'rb') as file:
    indices_top_5_directors_each_genre = pickle.load(file)

with open('Vipul/df_top5_productions.pkl', 'rb') as file:
    df_top5_productions = pickle.load(file)

with open('Vipul/df_content.pkl', 'rb') as file:
    df_content = pickle.load(file)

with open('Vipul/df_genre_counts.pkl', 'rb') as file:
    df_genre_counts = pickle.load(file)

genrelistcopy = ['Action & Adventure', 'Comedy', 'Drama', 'Science Fiction & Fantasy', 'Romance', 'Classics', 'Mystery & Suspense', 'Art House & International', 'Horror', 'Documentary', 'Special Interest', 'Television', 'Cult Movies']


genrelist = ['Action & Adventure', 'Comedy', 'Drama', 'Science Fiction & Fantasy', 'Romance', 'Classics', 'Kids & Family', 'Mystery & Suspense', 'Western', 'Art House & International', 'Horror', 'Faith & Spirituality', 'Animation', 'Documentary', 'Special Interest', 'Musical & Performing Arts', 'Sports & Fitness', 'Television', 'Cult Movies', 'Anime & Manga', 'Gay & Lesbian', '']

@app.callback(
    Output('genre-line-chart', 'figure'),
    [Input('genre-dropdown', 'value')]
)
def update_line_chart(selected_genres):
    traces = []
    for genre in selected_genres:
        genre_df = time_traces[genre]
        trace = go.Scatter(
            x=genre_df['year'],
            y=genre_df['positive_reviews'],
            mode='lines',
            name=genre
        )
        traces.append(trace)
    layout = go.Layout(
        title={'text':'Positive Reviews Over Years', 'font': {'color': 'white'}},
        xaxis={'title': 'Year', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},  # Set x-axis font color to white
        yaxis={'title': 'Positive Reviews', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},  # Set y-axis font color to white
        hovermode='closest',
        plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)',
        legend={'font': {'color': 'white'}}
    )
    return {'data': traces, 'layout': layout}

@app.callback(
    Output('genre-graph', 'figure'),
    [
        Input('genre-dropdown3', 'value')
    ]
)
def update_genre_graph(selected_genre):
    num_top_directors = 7
    directors = dictionary_top_5_directors_each_genre[selected_genre + 'Ratio'][:num_top_directors]
    top_directors_pve = df_top_critic_pntve.loc[indices_top_5_directors_each_genre[selected_genre + 'Ratio'][:num_top_directors], selected_genre + 'P'].tolist()
    top_directors_nve = df_top_critic_pntve.loc[indices_top_5_directors_each_genre[selected_genre + 'Ratio'][:num_top_directors], selected_genre + 'N'].tolist()
    
    fig = go.Figure(data=[
        go.Bar(name='Positive Reviews', x=directors, y=top_directors_pve,marker_color='#26D7FF'),
        go.Bar(name='Negative Reviews', x=directors, y=top_directors_nve,marker_color='#FFFFFF')
    ])
    fig.update_layout(
        title={'text':f'Reviews for Top {num_top_directors} Directors in {selected_genre}', 'font': {'color': 'white'}},
        barmode='group',
        xaxis={'title': 'Directors', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},
        yaxis={'title': 'Num of Reviews', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},
        plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)',
        legend={'font': {'color': 'white'}}
    )
    fig.update_layout(sliders=[{
        'len': 0.9,  # Adjusts the length of the slider
        'x': 0.1,  # Positioning of the slider
        'y': 0,  # Positioning of the slider
        'active': 0,
        'currentvalue': {
            'xanchor': 'right',
            'prefix': 'Year:',
            'font': {'size': 20, 'color': 'white'}  # Set font color to white
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},  # Transition effect
        'bgcolor': '#26D7FF',  # Background color of the slider, set to a dark grey
        'borderwidth': 2,  # Border width of the slider
        'bordercolor': 'white',  # Border color of the slider
        'pad': {'b': 10, 't': 100},  # Padding
        'currentvalue': {'visible': True},  # Shows the current value
        'font': {'color': 'white'}  # Set slider font color to white
    }])
    return fig

@app.callback(
    Output('actor-bar-chart', 'figure'),
    [Input('genre-dropdown2', 'value')]
)
def update_bar_chart(selected_genre):
    num_top_directors = 7
    # Get DataFrame of top actors for selected genre
    top_actors_df = top_actors_by_genre[selected_genre]
    top_actors_pve = df_top_critic_pntve.loc[indices_top_5_directors_each_genre[selected_genre + 'Ratio'][:num_top_directors], selected_genre + 'P'].tolist()
    top_actors_nve = df_top_critic_pntve.loc[indices_top_5_directors_each_genre[selected_genre + 'Ratio'][:num_top_directors], selected_genre + 'N'].tolist()
    
    
    # Create bar chart data
    data = [
        go.Bar(name='Positive Reviews', x=top_actors_df['actors'], y=top_actors_df['positive_reviews'], marker_color="#26D7FF"),
        go.Bar(name='Negative Reviews', x=top_actors_df['actors'], y=top_actors_df['negative_reviews'], marker_color="#FFFFFF")
    ]
    
    # Create bar chart layout with dynamic title
    layout = go.Layout(
        title={'text': f'Top 7 Actors in {selected_genre}', 'font': {'color': 'white'}},  # Dynamic title based on selected genre
        barmode='group',  # Display bars side by side
        xaxis={'title': 'Actors', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},
        yaxis={'title': 'Number of Reviews', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},
        plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)',
        legend={'font': {'color': 'white'}}
    )
    
    # Create bar chart figure
    figure = {'data': data, 'layout': layout}
    
    return figure

import numpy as np

@app.callback(
    Output('rating-pie-chart', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_pie_chart(selected_company):
    # Filter data for the selected production company
    selected_data = df_top5_productions[df_top5_productions['production_company'] == selected_company]
    
    # Extract ratings and corresponding values
    ratings = ['PG', 'R', 'NR', 'G', 'PG-13', 'NC17']
    values = [selected_data[rating].values[0] for rating in ratings]
    
    # Generate a range of colors from #26D7FF to #FFFFFF
    color_start = np.array([0x08, 0x68, 0xB9])  # RGB values for #26D7FF
    color_end = np.array([0xFF, 0xFF, 0xFF])    # RGB values for #FFFFFF
    num_slices = len(ratings)
    colors = [(color_start + (i / (num_slices - 1)) * (color_end - color_start)).astype(int).tolist() for i in range(num_slices)]
    colors = ['#%02x%02x%02x' % tuple(color) for color in colors]

    # Create pie chart using Plotly
    fig = go.Figure(data=[go.Pie(labels=ratings, values=values, marker=dict(colors=colors))])
    fig.update_layout(
        title={'text': f'Content Ratings Distribution for {selected_company}', 'font': {'color': 'white'}},
        plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)', 
        legend={'font': {'color': 'white'}}
    )
    fig.update_traces(textposition='outside', textinfo='percent+label',textfont=dict(color='white'))
    
    return fig




@app.callback(
    Output('genre-treemapV', 'figure'),
    [Input('genre-dropdownV', 'value')]
)
def update_genre_treemap(selected_genre):
    selected_row = df_content[df_content['Genre'] == selected_genre]
    
    fig = px.treemap(
        selected_row.melt(id_vars='Genre', var_name='Rating', value_name='Count'),
        path=['Genre', 'Rating'],
        values='Count',
        color='Count',
        color_continuous_scale='ice',
        # title={'text':f'Treemap for {selected_genre}', 'font': {'color': 'white'}},
        title="hello"
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)', legend={'font': {'color': 'white'}},
                     title={'text':f'Treemap for {selected_genre}', 'font': {'color': 'white'}})
    return fig



# Function to generate word cloud image and convert it to base64 string
def generate_wordcloud_image():
    # Function to return a fixed color
    def fixed_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
        return '#26D7FF'  # Set color to #26D7FF (light blue)

    # Generate word cloud based on the 'Genre' column weighted by 'Count'
    wordcloud = WordCloud(width=400, height=200, background_color=None, mode='RGBA', color_func=fixed_color_func).generate_from_frequencies(dict(zip(df_genre_counts['Genre'], df_genre_counts['Count'])))

    # Create matplotlib figure with transparent background
    plt.figure(figsize=(10, 6), facecolor=(1, 1, 1, 0))  # RGBA tuple with transparent background
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  # Turn off axis
    
    # Convert plot to image bytes
    image_stream = BytesIO()
    plt.savefig(image_stream, format='png', transparent=True)  # Save with transparent background
    image_stream.seek(0)
    img_str = base64.b64encode(image_stream.getvalue()).decode()
    img_src = 'data:image/png;base64,' + img_str
    
    plt.close()  # Close plot to prevent display
    
    return img_src

In [6]:
figV1.update_layout(
    xaxis=dict(showgrid=False, linecolor='black', tickfont=dict(color='white'),title=dict(font=dict(color='white'))),  # Remove vertical grid lines and set x-axis line color to black
    yaxis=dict(showgrid=True, gridcolor='black', tickfont=dict(color='white'),title=dict(font=dict(color='white'))),  # Show y-axis grid lines and set color to black
    title=dict(font=dict(color='white')),  # Set title font color to white
    legend=dict(font=dict(color='white')),  # Set legend font color to white
    sliders=[{
        'len': 0.9,  # Adjusts the length of the slider
        'x': 0.1,  # Positioning of the slider
        'y': 0,  # Positioning of the slider
        'active': 0,
        'currentvalue': {
            'xanchor': 'right',
            'prefix': 'Year:',
            'font': {'size': 20, 'color': 'white'}  # Set font color to white
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},  # Transition effect
        'bgcolor': '#26D7FF',  # Background color of the slider, set to a dark grey
        'borderwidth': 2,  # Border width of the slider
        'bordercolor': 'white',  # Border color of the slider
        'pad': {'b': 10, 't': 100},  # Padding
        'currentvalue': {'visible': True},  # Shows the current value
        'font': {'color': 'white'}  # Set slider font color to white
    }]
)

colors = ['#A3EEFF', '#26D7FF', '#FFFFFF', '#ECECEC']  # Define colors for each set of bars

for i, color in enumerate(colors):
    figV1.data[i].marker.color = color

for trace in figV1.data:
    trace.marker.line.color = 'rgba(0, 0, 0, 0)'  # Set border color to transparent


# Anshul and Ruby tab

In [7]:
with open('Anshul/donut.pkl', 'rb') as file:
    A1 = pickle.load(file)
A1.update_layout(
    colorway=['#FF5733', '#FFC300', '#DAF7A6', '#FF5733', '#C70039', '#581845']
)


with open('Anshul/trend.pkl', 'rb') as file:
    A2 = pickle.load(file)

with open('Anshul/worldmap.pkl', 'rb') as file:
    A3 = pickle.load(file)
A3.update_layout(paper_bgcolor='red',plot_bgcolor="pink")


with open('Anshul/top10moviesOverall.pkl', 'rb') as file:
    A4 = pickle.load(file)

with open('Anshul/top10tvshowOverall.pkl', 'rb') as file:
    A5 = pickle.load(file)

with open('Anshul/A6.pkl', 'rb') as file:
    A6 = pickle.load(file)

# with open('Anshul/A7.pkl', 'rb') as file:
#     A7 = pickle.load(file)

with open('Anshul/top10moviesYearwise.pkl', 'rb') as file:
    A8 = pickle.load(file)

with open('Anshul/top10tvshowYearwise.pkl', 'rb') as file:
    A9 = pickle.load(file)

with open('Anshul/top_movies_by_genreN.pkl', 'rb') as file:
    top_movies_by_genre = pickle.load(file)

with open('Anshul/top_series_by_genreN.pkl', 'rb') as file:
    top_series_by_genre = pickle.load(file)

with open('Anshul/data.pkl', 'rb') as file:
    data = pickle.load(file)

with open('Anshul/gen1.pkl', 'rb') as file:
    gen1 = pickle.load(file)
    
# load pickle dataset for dash plot
    
#callback

In [8]:
# Initialize dropdown options with 'Action' genre as default
dropdown_options = [{'label': 'Action', 'value': 'Action'}]

# Iterate over each genre
for genre in top_series_by_genre.keys():
    dropdown_options.append({'label': genre, 'value': genre})

# Create an initial selection
initial_genre = 'Action'

@app.callback(
    Output('genre-treemapAR', 'figure'),
    [Input('genre-dropdownAR', 'value')]
)

# Function to update the figure based on the selected genre
def update_figure(selected_genre):
    top_series = top_series_by_genre[selected_genre]  # Get top series for the selected genre
    top_movies = top_movies_by_genre[selected_genre]  # Get top movies for the selected genre
    
    top_movie = [f"{title}: {rating}" for title, rating in zip(top_movies['title'], top_movies['rating'])]

    top_series = [f"{title}: {rating}" for title, rating in zip(top_series['title'], top_series['rating'])]
    
    # Create treemap
    fig9 = go.Figure(go.Treemap(
        labels=["All", "Movies"]  + top_movie + ["Series"] + top_series ,
        parents=[""] + ["All"] + (["Movies"]*len(top_movie))  + ["All"] +  (["Series"]*len(top_series)) ,
        textfont=dict(
        size=12,  # Set font size
        ),
        marker_colorscale = 'blues',

    ))

    # Update layout
    fig9.update_layout(
        title=f'Top 10 Movies and Series in the {selected_genre} Genre Based on Ratings',
        treemapcolorway=['rgb(158,202,225)'],  # Set treemap color
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='white')
    )

    return fig9

# Define callback to update the figure based on dropdown selection

def update_genre_treemap(selected_genre):
    return update_figure(selected_genre)

In [9]:
data_2010 = data[data['releaseYear'] == 2010]

# Calculate the average rating for each genre in 2010
avg_rating_by_genre = data_2010.groupby('genre')['rating'].mean().reset_index()

# Create the bubble graph
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=avg_rating_by_genre['genre'],
    y=avg_rating_by_genre['rating'],
    mode='markers',
    marker=dict(
        size=avg_rating_by_genre['rating'] * 10,  # Adjust the size of bubbles based on rating
        color=avg_rating_by_genre['rating'],  # Color bubbles based on rating
        colorscale='Blues',  # Choose a colorscale
        showscale=True  # Show a color scale legend
    )
))

# Update layout
fig.update_layout(
    title='Average Ratings by Genre in 2010',
    xaxis=dict(title='Genre'),
    yaxis=dict(title='Average Rating'),
    hovermode='closest'
)

# Define the Dash layout
# Define callback to update the bubble graph based on the selected year
@app.callback(
    Output('bubble-graph', 'figure'),
    [Input('year-slider', 'value')]
)
def update_bubble_graph(selected_year):
    data_selected_year = data[data['releaseYear'] == selected_year]
    avg_rating_by_genre_selected_year = data_selected_year.groupby('genre')['rating'].mean().reset_index()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=avg_rating_by_genre_selected_year['genre'],
        y=avg_rating_by_genre_selected_year['rating'],
        mode='markers',
        marker=dict(
        size=avg_rating_by_genre_selected_year['rating'] * 4,
            color=avg_rating_by_genre_selected_year['rating'],
            colorscale='Blues',
            showscale=True
        )
    ))

    fig.update_layout(
        title=f'Average Ratings by Genre in {selected_year}',
        xaxis=dict(title='Genre'),
        yaxis=dict(title='Average Rating'),
        hovermode='closest',
        plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)',
        font = dict(  color = 'white')
        )

    return fig

In [10]:
# Define callback to update histogram based on dropdown selection
@app.callback(
    Output('movie-histogramAR', 'figure'),
    [Input('country-dropdownAR', 'value'),
     Input('genre-dropdownAR1', 'value')]
)
def update_histogram(selected_country, selected_genre):
    # Filter data for the selected country and genre, and contentType is movie
    country_genre_data = gen1[(gen1['region'] == selected_country) & (gen1['genre'] == selected_genre) & (gen1['contentType'] == 'movie')]
    
    # Group data by movie title and calculate the average rating
    avg_rating_per_movie = country_genre_data.groupby('title')['rating'].mean().reset_index()
    
    # Select the top 10 movies based on average rating
    top_10_movies = avg_rating_per_movie.nlargest(10, 'rating')

    # Create histogram
    fig11 = go.Figure()
    fig11.add_trace(go.Bar(x=top_10_movies['title'], y=top_10_movies['rating'],marker=dict(color="#01abd2"),))
    fig11.update_layout(title=f"Top 10 {selected_genre.capitalize()} Movies in {selected_country} (Based on Ratings)",
                        xaxis_title="Movie Title", 
                        yaxis_title="Average Rating",
                        
                        plot_bgcolor='rgba(0,0,0,0)', 
                        paper_bgcolor='rgba(0,0,0,0)',
                        font=dict(color='white'),
#                         marker=dict(color="#01abd2")
                        
                       )

    return fig11

# Web APP

In [11]:
image_path = 'assets/background.jpg'

In [12]:
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold',
    'font-size' : '30px',
    'background-color':'black',
    'color' : 'white'
}

selected_tab_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#303030',
    'color': 'white',
    'padding': '6px',
    'font-size':'30px'
}

app.layout = html.Div([
    dcc.Tabs(
        id='tabs',
        value='tab-3',
        children=[
            dcc.Tab(label='Home', value='tab-3',selected_style = {'backgroundColor': 'black','color':'white','font-size':'30px'}, style={'backgroundColor': '#303030', 'color': 'white','font-size':'30px'}),
            dcc.Tab(label='WorldWide Flick Fusion', value='tab-2',selected_style = {'backgroundColor': 'black','color':'white','font-size':'30px'}, style={'backgroundColor': '#303030', 'color': 'white','font-size':'30px'}),
            dcc.Tab(label='Opinion Odyssey', value='tab-4',selected_style = {'backgroundColor': 'black','color':'white','font-size':'30px'}, style={'backgroundColor': '#303030', 'color': 'white','font-size':'30px'}),
            dcc.Tab(label='Screen Evolution', value='tab-1',selected_style = {'backgroundColor': 'black','color':'white','font-size':'30px'}, style={'backgroundColor': '#303030', 'color': 'white','font-size':'30px'}),
            # dcc.Tab(label='Tab 4', value='tab-5',selected_style = {'backgroundColor': 'black','color':'white','font-size':'30px'}, style={'backgroundColor': '#303030', 'color': 'white','font-size':'30px'}),
            
 
        ],
          # Apply custom styling directly to the Tabs component
    ),
    html.Div(id='tabs-content')
],style={'background-color':'black','height':'110px', 'font-family': 'Arial, sans-serif'} ) # Setting background color directly in layout



@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
           html.H1("Television Shows and Movies Across the Decades", style={'text-align': 'center', 'color': '#ffa500', 'margin-bottom': '20px'}),
    html.Div([
        html.Div([
            dcc.Graph(id='racing_chart_movies',figure=fig_racing_chart_movies),
        ], className='six columns', style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='racing_chart_tvseries',figure=fig_racing_chart_tvseries),
        ], className='six columns', style={'margin-bottom': '20px'}),
    ], className='row'),
    html.H1("Select a Genre", style={'text-align': 'center', 'color': '#ffa500', 'margin-bottom': '20px'}),
    html.Div([
        dcc.Dropdown(
            id='genre-dropdown',
            options=[{'label': genre, 'value': genre} for genre in tv_series_dict.keys()],
            value='Action',  # Default value
            style={'width': '40%', 'margin': 'auto', 'text-align': 'center', 'background-color': 'white', 'color': 'black', 'border': 'none'},
            clearable=False 
        )
    ], style={'text-align': 'center', 'margin-bottom': '20px'}),
    html.Div([
        html.Div([
            dcc.Graph(id='fig1'),
        ], className='six columns', style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='fig2'),
        ], className='six columns', style={'margin-bottom': '20px'}),
    ], className='row'),
    html.Div([
        html.Div([
            dcc.Graph(id='fig3'),
        ], className='six columns', style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='fig4'),
        ], className='six columns', style={'margin-bottom': '20px'}),
    ], className='row'),
    html.Div([
        html.Div([
            dcc.Graph(id='fig5'),
        ], className='six columns', style={'margin-bottom': '20px'}),
    ], className='row')
], style={'margin': 'auto', 'max-width': '1200px', 'background-color': '#0d1117', 'padding': '200px', 'color': 'white', 'outline': 'none','background-image': 'linear-gradient(to right, black, darkgrey)',
                  'background-size': '400% 400%',},
                  id='fade-in-div1')

    elif tab == 'tab-2':
        return html.Div([
            html.Div([
                dcc.Graph(id='A1', 
                          figure=A1.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
                         ),
                dcc.Graph(
                    id='A2', 
                    figure=A2.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
                ),
            ], style={
                'display':'flex',
                'justify-content':'space-between',
                'flex-wrap':'wrap'
            } ),
            dcc.Graph(
                id='A3',
                figure=A3.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            ),
            html.Div([
                dcc.Graph(id='A4', 
                          figure=A4.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'),
                          style={'width':'45%','background-color':'#303030','border-radius':'10px','box-shadow':'3px 2px 2px #d1d6d2'}
                         ),
                dcc.Graph(
                    id='A5', 
                    figure=A5.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'),
                    style={'width':'45%','background-color':'#303030','border-radius':'10px','box-shadow':'3px 2px 2px #d1d6d2'}
                ),
            ], style={
                'display':'flex',
                'justify-content':'space-around',
                'flex-wrap':'wrap',
                'margin':'15px'
            } ),
            html.Div([
                dcc.Graph(id='A8', 
                          figure=A8.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'),
                          style={'width':'45%','background-color':'#303030','border-radius':'10px','box-shadow':'3px 2px 2px #d1d6d2'}
                         ),
                dcc.Graph(
                    id='A9', 
                    figure=A9.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'),
                    style={'width':'45%','background-color':'#303030','border-radius':'10px','box-shadow':'3px 2px 2px #d1d6d2'}
                ),

            ], style={
                'display':'flex',
                'justify-content':'space-around',
                'flex-wrap':'wrap',
                'margin':'15px',
                'margin-top':'30px',
                'margin-bottom':'30px'
            } ),
            html.Div("Rating Analysis", style={'textAlign': 'center', 'color': 'white', 'fontSize': 40, 'marginTop': 30}),
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdownAR',
                    options=dropdown_options,
                    value=initial_genre,
                    clearable=False,
                    style={
                        'color': 'grey',  # Set font color to white
                        'backgroundColor': '#d6d6d6',  # Set background color to black
                        'height': '40px' , 'width':'500px','display': 'flex',  'justify-content': 'center','align-items': 'center',
                    }
                ),
                dcc.Graph(
                    id='genre-treemapAR',
                    style={'height':'600px','width':'1200px','display': 'flex',  'justify-content': 'center','align-items': 'center','background-color': 'rgba(0, 0, 0, 0)','color':'black'} 
                )
            ],
            style={
                'display':'flex',
                'flex-direction':'column',
                'justify-content':'center',
                'align-items':'center',
                'flex-wrap':'wrap',
                'margin':'15px',
                'margin-top':'50px',
                'margin-bottom':'30px',
                'margin-right':'200px',
                'width':'100%',
                # 'background-color':'black',
            }),
            html.Div([
                dcc.Graph(id='bubble-graph', figure=fig),
                html.Div(
                    dcc.Slider(
                        id='year-slider',
                        min=2010,
                        max=2022,
                        value=2010,
                        marks={str(year): str(year) for year in range(2010, 2023)},
                        # marks = ,
                        step=None,
                    ),
                    style={'width': '80%', 'margin': '0 auto', 'color': 'black','background-color': 'rgba(0, 0, 0, 0)'}
                )
            ]),
            html.Div([
                dcc.Dropdown(
                    id='country-dropdownAR',
                    options=[{'label': country, 'value': country} for country in gen1['region'].unique()],
                    value=gen1['region'].unique()[0],  # Set default value to the first country
                    clearable=False,
                     style={
                        'color': 'grey',  # Set font color to white
                        'backgroundColor': '#d6d6d6',  # Set background color to black
                        'height': '40px' , 'width':'500px','display': 'flex',  'justify-content': 'center','align-items': 'center',
                    }
                ),
                dcc.Dropdown(
                    id='genre-dropdownAR1',
                    options=[{'label': genre, 'value': genre} for genre in gen1['genre'].unique()],
                    value=gen1['genre'].unique()[0],  # Set default value to the first genre
                    clearable=False,
                     style={
                        'color': 'grey',  # Set font color to white
                        'backgroundColor': '#d6d6d6',  # Set background color to black
                         'height': '40px' , 'width':'500px','display': 'flex',  'justify-content': 'center','align-items': 'center',
                    }
                ),
                dcc.Graph(id='movie-histogramAR',
                         style={
                            'width':'1000px'
                         })
            ],style={
                'display':'flex',
                'flex-direction':'column',
                'justify-content':'center',
                'align-items':'center',
                'flex-wrap':'wrap',
                'margin':'15px',
                'margin-top':'50px',
                'margin-bottom':'30px',
                'margin-right':'200px',
                'width':'100%',
                # 'background-color':'black',
            }),
        ],style={'background-image': 'linear-gradient(to right, black, darkgrey)',
                  'background-size': '400% 400%',
                },
        id='fade-in-div2'
    ),
        
    elif tab == 'tab-3':
       return html.Div([
            html.H1("Cine Insights: Decoding the Secrets of Cinema", style={
                "font-size": "50px", "color": "#0464b4", "font-style": "italic","text-shadow":"2px 2px 2px grey", 'text-align': 'center'
            }),
            html.Img(src=image_path , style={'position':'absolute', 'width':'100%','height':'400' , 'opacity':'0.3','z-index':'-1','top':'0px','left':'0px'}),
        ], id='fade-in-div3', style={
            'margin-top':'10px', 'color': 'white', 'position': 'relative','justify-content':'center','align-items':'center',
        })

    elif tab == 'tab-4':
        return html.Div([
            html.H3('Tab 4 content'),

            html.Div([
                html.Img(id='wordcloud-image', src=generate_wordcloud_image(), style={'width': '60%', 'height': '40%'})
                ],  style={'text-align': 'center'}),
    
            # First graph
            html.Div([
                dcc.Graph(
                    id='tab4-graph',
                    figure=figV1.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')  # Assuming figV1 is defined elsewhere
                )
            ]),
    
            # Second graph with dropdown
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdown',
                    options=[{'label': html.Span(genre), 'value': genre} for genre in time_traces.keys()],
                    value=['Animation'],  # Default selected genres
                    multi=True,  # Allow multiple selections
                    className='custom-dropdown',  # Add custom CSS class
                    style = {'background-color': '#000000'}
                ),
                dcc.Graph(id='genre-line-chart')
            ]),

    
            # Third graph with dropdown
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdown2',
                    options=[{'label': html.Span(genre, style={'color': '#26D7FF'}), 'value': genre} for genre in top_actors_by_genre.keys()],
                    value=list(top_actors_by_genre.keys())[0],  # Default value for dropdown (first genre)
                    style={'width': '50%', 'background-color': 'rgba(0,0,0,0)', 'color': 'black'},
                    clearable=False,
                ),
                dcc.Graph(id='actor-bar-chart')
            ]),
    
            # Fourth graph with dropdown, slider, and radio items
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdown3',
                    options=[{'label': html.Span(genre, style={'color': '#26D7FF'}), 'value': genre} for genre in genrelist],
                    value=genrelist[0],
                    clearable=False,
                    style={'width': '50%','background-color': 'rgba(0, 0, 0, 0)', 'color': '#000000'}
                ),
                dcc.Graph(id='genre-graph')
            ],style={'background-color': 'rgba(0, 0, 0, 0)'}),
    
            # Fifth graph with dropdown
            html.Div([
                dcc.Dropdown(
                    id='company-dropdown',
                    options=[{'label': html.Span(company, style={'color': '#26D7FF'}), 'value': company} for company in df_top5_productions['production_company']],
                    value=df_top5_productions['production_company'][0],  # Default value is the first production company
                    style = {'background-color': 'rgba(0, 0, 0, 0)', 'color': 'black'},
                    clearable=False,
                ),
                dcc.Graph(id='rating-pie-chart')
            ]),
    
            # Sixth graph with dropdown
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdownV',
                    options=[{'label': html.Span(genre, style={'color': '#26D7FF'}), 'value': genre} for genre in genrelistcopy],
                    value=genrelist[0],
                    style={'width': '50%', 'background-color': 'rgba(0, 0, 0, 0)', 'color': 'black'},
                    clearable=False,
                ),
                dcc.Graph(id='genre-treemapV')
            ])
        ],style={'background-image': 'linear-gradient(to right, black, darkgrey)',
                  'background-size': '400% 400%',}  , id='fade-in-div4')
        
        




In [14]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8009)

# http://127.0.0.1:8008/


C:\Users\ASUS\anaconda3\Lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of /assets/styles.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

